# wandb

This notebook is a further refinement of the model defined in `train.ipynb` that replaces TensorBoard with Spell's [Weights & Biases integration](https://spell.ml/docs/integrating_wandb/).

In [2]:
%pip install wandb

     |████████████████████████████████| 1.4 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 4.5 MB/s  eta 0:00:01
     |████████████████████████████████| 97 kB 9.2 MB/s  eta 0:00:01
     |████████████████████████████████| 70 kB 11.3 MB/s eta 0:00:01
  Created wheel for watchdog: filename=watchdog-0.10.3-py3-none-any.whl size=73871 sha256=741b4065a3d4764b97db76f899d13994c86cb05a1b31699910c0c79b3d34fa9c
  Stored in directory: /root/.cache/pip/wheels/27/21/35/9d1e531f9de5335147dbef07e9cc99d312525ba128a93d1225
  Created wheel for gql: filename=gql-0.2.0-py3-none-any.whl size=7628 sha256=60242d2653d874d898ab7e79a75e825327c167ff0e1a9106ad9a2198d88bf199
  Stored in directory: /root/.cache/pip/wheels/b6/9a/56/5456fd32264a8fc53eefcb2f74e24e99a7ef4eb40a9af5c905
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19189 sha256=a846aa49386f5295a6f1c58d609c846aaebc342006d37b87f08adcc020848df1
  Stored in directory: /root/.cache/pip/wheels/df/

In [4]:
%%writefile ../models/wandb_train.py
import torchvision
from torch.utils.data import DataLoader
import torch
from torch import nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import argparse
from spell.metrics import send_metric

import re
import os
if not os.path.exists("/spell/checkpoints/"):
    os.mkdir("/spell/checkpoints/")
writer = SummaryWriter('/spell/tensorboard/')

import wandb

# parser = argparse.ArgumentParser()
# parser.add_argument('--epochs', type=int, dest='epochs', default=50)
# parser.add_argument('--batch_size', type=int, dest='batch_size', default=32)

# parser.add_argument('--conv1_filters', type=int, dest='conv1_filters', default=32)
# parser.add_argument('--conv2_filters', type=int, dest='conv2_filters', default=64)
# parser.add_argument('--dense_layer', type=int, dest='dense_layer', default=512)

# parser.add_argument('--conv1_dropout', type=float, dest='conv1_dropout', default=0.25)
# parser.add_argument('--conv2_dropout', type=float, dest='conv2_dropout', default=0.25)
# parser.add_argument('--dense_dropout', type=float, dest='dense_dropout', default=0.5)

# parser.add_argument('--from_checkpoint', type=str, dest='from_checkpoint', default="")

# args = parser.parse_args()

# Used for testing purposes.
class Args:
    def __init__(self):
        self.epochs = 50
        self.batch_size = 32
        self.conv1_filters = 32
        self.conv2_filters = 64
        self.dense_layer = 512
        self.conv1_dropout = 0.25
        self.conv2_dropout = 0.25
        self.dense_dropout = 0.5
        self.from_checkpoint = False
args = Args()

transform_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    # torchvision.transforms.Lambda(lambda x: torch.tensor(np.array(x).reshape((3, 32, 32)) / 255, dtype=torch.float)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
transform_test = torchvision.transforms.Compose([
    # torchvision.transforms.Lambda(lambda x: torch.tensor(np.array(x).reshape((3, 32, 32)) / 255, dtype=torch.float)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),    
])

download = not os.path.exists("/mnt/cifar10/")
if download:
    print("CIFAR10 dataset not on disk, downloading...")
else:
    print("CIFAR10 dataset is already on disk! Skipping download.")

train_dataset = torchvision.datasets.CIFAR10("/mnt/cifar10/", train=True, transform=transform_train, download=download)
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)
# download=False because the train and test sets are downloaded simultaneously
test_dataset = torchvision.datasets.CIFAR10("/mnt/cifar10/", train=False, transform=transform_test, download=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

class CIFAR10Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn_block_1 = nn.Sequential(*[
            nn.Conv2d(3, args.conv1_filters, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(args.conv1_filters, args.conv2_filters, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(args.conv1_dropout)
        ])
        self.cnn_block_2 = nn.Sequential(*[
            nn.Conv2d(args.conv2_filters, args.conv2_filters, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(args.conv2_filters, args.conv2_filters, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(args.conv2_dropout)
        ])
        self.flatten = lambda inp: torch.flatten(inp, 1)
        self.head = nn.Sequential(*[
            nn.Linear(args.conv2_filters * 8 * 8, args.dense_layer),
            nn.ReLU(),
            nn.Dropout(args.dense_dropout),
            nn.Linear(args.dense_layer, 10)
        ])
    
    def forward(self, X):
        X = self.cnn_block_1(X)
        X = self.cnn_block_2(X)
        X = self.flatten(X)
        X = self.head(X)
        return X

clf = CIFAR10Model()

if args.from_checkpoint:
    if args.from_checkpoint == "latest":
        start_epoch = max([int(re.findall("[0-9]{1,2}", fp)[0]) for fp in os.listdir("/mnt/checkpoints/")])
    else:
        start_epoch = args.from_checkpoint
    clf.load_state_dict(torch.load(f"/mnt/checkpoints/epoch_{start_epoch}.pth"))
    print(f"Resuming training from epoch {start_epoch}...")
else:
    start_epoch = 1

clf.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(clf.parameters(), lr=0.0001, weight_decay=1e-6)
def test(epoch, num_epochs):
    losses = []
    n_right, n_total = 0, 0
    clf.eval()
    
    for i, (X_batch, y_cls) in enumerate(test_dataloader):
        with torch.no_grad():
            y = y_cls.cuda()
            X_batch = X_batch.cuda()

            y_pred = clf(X_batch)
            loss = criterion(y_pred, y)
            losses.append(loss.item())
            _, y_pred_cls = y_pred.max(1)
            n_right, n_total = n_right + (y_pred_cls == y_cls.cuda()).sum().item(), n_total + len(X_batch)
    
    val_acc = n_right / n_total
    val_loss = np.mean(losses)
    
    send_metric("val_loss", val_loss)
    send_metric("val_acc", val_acc)
    wandb.log({"val_loss": val_loss, "val_acc": val_acc})
    print(
        f'Finished epoch {epoch}/{num_epochs} avg val loss: {val_loss:.3f}; median val loss: {np.median(losses):.3f}; '
        f'val acc: {val_acc:.3f}.'
    )

def train():
    clf.train()
    NUM_EPOCHS = args.epochs
    
    wandb.init()
    wandb.config.update(args)
    wandb.watch(clf)
    
    for epoch in range(start_epoch, NUM_EPOCHS + 1):
        losses = []

        for i, (X_batch, y_cls) in enumerate(train_dataloader):
            optimizer.zero_grad()

            y = y_cls.cuda()
            X_batch = X_batch.cuda()

            y_pred = clf(X_batch)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()

            train_loss = loss.item()
            if i % 200 == 0:
                print(
                    f'Finished epoch {epoch}/{NUM_EPOCHS}, batch {i}. loss: {train_loss:.3f}.'
                )
                send_metric("train_loss", train_loss)
                wandb.log({"train_loss": train_loss})
            losses.append(train_loss)

        print(
            f'Finished epoch {epoch}. '
            f'avg loss: {np.mean(losses)}; median loss: {np.median(losses)}'
        )
        test(epoch, NUM_EPOCHS)    
        if epoch % 5 == 0:
            torch.save(clf.state_dict(), f"/spell/checkpoints/epoch_{epoch}.pth")
    
    torch.save(clf.state_dict(), f"/spell/checkpoints/epoch_{NUM_EPOCHS}.pth")

if __name__ == "__main__":
    train()

Writing ../models/wandb_train.py
